# Federated Learning Generalizability Study: Multi-Model Comparison

## Overview
This notebook implements and compares three federated learning approaches for tomato disease classification using spectral data. The study evaluates model generalization across different federated learning strategies.

## Models Implemented
1. **Logistic Regression** - Using SGDClassifier with log loss
2. **SVM (Support Vector Machine)** - Using SGDClassifier with hinge loss
3. **Neural Network** - Using PyTorch with cross-entropy loss

## Evaluation Framework
- **Client Configuration**: 5 clients total (4 for training, 1 held out for testing)
- **Cross-Validation**: 20-fold cross-validation for robust statistical evaluation
- **Feature Engineering**: B-spline basis transformation from raw spectral data
- **Evaluation Strategies**:
  - Federated Learning (FedAvg)
  - Personalized Federated Learning
  - Local Training
  - Local Average
  - Centralized Training

In [ ]:
# Import all required libraries for all three approaches
!pip install -q flwr[simulation]
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Federated learning imports
import flwr as fl
from flwr.common import Context, Parameters, ndarrays_to_parameters
from flwr.client import NumPyClient, Client, ClientApp
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from flwr.simulation import run_simulation
from flwr.server.strategy import FedAvg, FedProx, FedAdam, FedOpt

# ML imports
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import hinge_loss, accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from scipy.interpolate import BSpline, make_lsq_spline

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim

# Configuration
NUM_PARTITIONS = 4  # 4 training clients + 1 test client = 5 total
basisnum = 10
NUM_FOLDS = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Environment setup complete!")
print(f"PyTorch device: {device}")

## Data Processing Pipeline

### Data Loading and Preprocessing
```python
# Load multiple CSV files and combine them
dataframes = []
labels = []
all_data = []

for file in uploaded:
    df = pd.read_csv(file, header=None)
    df = df.T  # Transpose data
    df.columns = df.iloc[0]  # Use first row as column headers
    df = df.drop(df.index[0])  # Remove header row
    df = df.apply(pd.to_numeric, errors='coerce')  # Convert to numeric
    label = 1 if "diseased" in file.lower() else 0  # Binary classification
    df["label"] = label
    all_data.append(df)
```
###Data Structure:
- Input: CSV files containing spectral data
- Labels: Binary classification (0=healthy, 1=diseased)
- Features: Spectral measurements (300 wavelengths)

###Feature Extraction
```python
df_all = pd.concat(all_data, ignore_index=True)
feature_cols = df_all.columns[2:-1]  # Select relevant feature columns
X = df_all[feature_cols].values  # Feature matrix (N, 300)
y = df_all['label'].values  # Target labels (N,)
```

In [ ]:
# Data loading section - replace with your actual data loading
from google.colab import files
uploaded = files.upload()

dataframes = []
labels = []
all_data = []

for file in uploaded:
    df = pd.read_csv(file, header=None)
    df = df.T
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df = df.apply(pd.to_numeric, errors='coerce')
    print(f"{file}: shape = {df.shape}")
    label = 1 if "diseased" in file.lower() else 0
    df["label"] = label
    all_data.append(df)

df_all = pd.concat(all_data, ignore_index=True)
feature_cols = df_all.columns[2:-1]  # remove unneeded columns

X = df_all[feature_cols].values  # 2D array (N, 300)
y = df_all['label'].values  # labels (N,)
print("X shape:", X.shape)
print("y shape:", y.shape)

##Feature Engineering

###B-spline Basis Transformation
Purpose: Reduces dimensionality from 300 spectral bands to configurable number of B-spline basis functions while preserving spectral characteristics and meets federated learning requirements.

In [ ]:
def compute_knot_vector(num_basis, spline_order):
    num_total_knots = num_basis + spline_order + 1
    num_internal_knots = num_total_knots - 2 * (spline_order + 1)
    internal_knots = np.linspace(0, 1, num_internal_knots + 2)[1:-1]
    t = np.concatenate((
        np.repeat(0.0, spline_order + 1),
        internal_knots,
        np.repeat(1.0, spline_order + 1)
    ))
    return t

def fit_bspline_basis(X, num_basis=10, spline_order=3):
    N, D = X.shape
    x_domain = np.linspace(0, 1, D)
    t = compute_knot_vector(num_basis, spline_order)

    coeffs = np.zeros((N, num_basis))
    for i in range(N):
        y = X[i]
        spline = make_lsq_spline(x_domain, y, t, spline_order)
        coeffs[i] = spline.c

    return coeffs

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Client Data Preparation Functions

These act as a pipeline, calling the b-spline function and then standardizing the dataset as well. This can easily be called later.

In [ ]:
def preprocess_data(X_data, num_basis=basisnum, scaler=None):
    """Apply B-spline transformation and scaling in one step"""
    X_reduced = fit_bspline_basis(X_data, num_basis=num_basis, spline_order=3)
    if scaler is None:
        scaler = StandardScaler()
        return scaler.fit_transform(X_reduced), scaler
    return scaler.transform(X_reduced), scaler

# Refactored client data preparation
def prepare_client_data(X_train_partitions_raw, X_test_partitions_raw,
                       y_train_partitions, y_test_partitions):
    """Prepare all client data with preprocessing"""
    client_train_data, client_val_data = {}, {}

    for cid in range(NUM_PARTITIONS):
        X_train_scaled, scaler = preprocess_data(X_train_partitions_raw[cid])
        X_test_scaled, _ = preprocess_data(X_test_partitions_raw[cid], scaler=scaler)

        client_train_data[cid] = (X_train_scaled, y_train_partitions[cid])
        client_val_data[cid] = (X_test_scaled, y_test_partitions[cid])

    return client_train_data, client_val_data

##Logistic Regression Setup

This code sets up a basic federated learning workflow using the Flower library and scikit-learn's SGDClassifier (log loss).

Parameter management: Functions get, set, and average the model's weights across different clients.  
Model utilities: Functions for incremental training (partial_fit) and evaluating with standard metrics (loss, accuracy, precision, recall, F1).  
FlowerClient: Each client handles its own data partition, receives and updates model parameters, trains locally, and reports metrics.  
Server functions: Aggregate client metrics, perform federated averaging, and orchestrate rounds.  
Configuration: Specifies number of rounds and system resources for clients.

In [ ]:
def get_parameters(model):
    return [model.coef_.copy(), model.intercept_.copy()]

def set_parameters(model, parameters):
    model.coef_ = parameters[0].copy()
    model.intercept_ = parameters[1].copy()

def average_parameters(parameter_list):
    avg_coef = np.mean([params[0] for params in parameter_list], axis=0)
    avg_intercept = np.mean([params[1] for params in parameter_list], axis=0)
    return [avg_coef, avg_intercept]

def train_model(model, X, y):
    if not hasattr(model, "coef_"):
        model.partial_fit(X, y, classes=np.array([0,1]))
    else:
        model.partial_fit(X, y)

def evaluate_model(model, X, y):
    preds = model.predict(X)
    y_prob = model.predict_proba(X)
    return {
        "loss": log_loss(y, y_prob, labels=model.classes_),
        "accuracy": accuracy_score(y, preds),
        "precision": precision_score(y, preds, average="binary", zero_division=0),
        "recall": recall_score(y, preds, average="binary", zero_division=0),
        "f1": f1_score(y, preds, average="binary", zero_division=0),
    }

def initialize_model(n_features):
    model = SGDClassifier(
        loss="log_loss",
        max_iter=20,
        tol=None,
        warm_start=True,
        learning_rate="optimal",
        eta0=0.01,
        random_state=42,
    )
    return model

class FlowerClient(NumPyClient):
    def __init__(self, partition_id: int, personalized=False):
        self.partition_id = partition_id
        self.model = initialize_model(X_train_full.shape[1])
        self.X_train, self.y_train = client_train_data[partition_id]
        self.X_val, self.y_val = client_val_data[partition_id]
        self.personalized = personalized
        self.model.partial_fit(self.X_train, self.y_train, classes=np.array([0, 1]))


    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        train_model(self.model, self.X_train, self.y_train)
        return get_parameters(self.model), len(self.X_train), {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        metrics = evaluate_model(self.model, self.X_val, self.y_val)
        # metrics = evaluate_model(self.model, self.X_train, self.y_train)
        loss = metrics.pop("loss", 0.0)
        return loss, len(self.y_val), metrics

final_fl_metrics = None
final_parameters = None

def get_evaluate_fn():
    def evaluate(server_round, parameters, config):
        all_metrics = []
        for cid in range(NUM_PARTITIONS):
            model = initialize_model(X_train_full.shape[1])
            X_val, y_val = client_val_data[cid]
            X_train, y_train = client_train_data[cid]
            set_parameters(model, parameters)
            model.classes_ = np.unique(y_train_full)
            metrics = evaluate_model(model, X_val, y_val)
            # metrics = evaluate_model(model, X_train, y_train)
            all_metrics.append(metrics)

        df_metrics = pd.DataFrame(all_metrics)
        avg_metrics = df_metrics.mean().to_dict()

        if server_round == 10:
            global final_fl_metrics, final_parameters
            final_fl_metrics = avg_metrics.copy()
            final_parameters = parameters

        return avg_metrics["loss"], {"accuracy": avg_metrics["accuracy"]}
    return evaluate

def server_fn(context):
    strategy = FedAvg(
        min_available_clients=NUM_PARTITIONS,
        evaluate_fn=get_evaluate_fn(),
        fit_metrics_aggregation_fn=None,
        evaluate_metrics_aggregation_fn=None
    )
    return ServerAppComponents(config=ServerConfig(num_rounds=10), strategy=strategy)

def client_fn(context):
    cid = context.node_config.get("partition-id", 0)
    return FlowerClient(cid).to_client()

client = ClientApp(client_fn=client_fn)
server = ServerApp(server_fn=server_fn)

backend_config = {"client_resources": {"num_cpus": 2, "num_gpus": 0}}


##Cross-Validation Evaluation of Federated and Baseline Models for LR
This script runs repeated train/test splits to compare federated, personalized, local, and centralized models.

- For each fold, the data is split into training, testing, and personalized subsets with stratification.
- Each partition is preprocessed and used to prepare client data.
- A federated simulation runs, aggregating client models.
- Evaluations are performed for:
federated learning (global model),
personalized federated (fine-tuned global model),
local-only models (trained on each client),
averaged local models (ensemble of locals),
and a centralized model (trained on pooled data).
- All results and summary statistics are saved for further analysis.

In [ ]:
results = {
    "federated": [],
    "personalized_federated": [],
    "local": [],
    "local_avg": [],
    "centralized": []
}

all_results_by_basis = {}
fold = 0

print(f"--- Basis {basisnum} ---")
for repeat in range(NUM_FOLDS):
    print(f"--- Fold {repeat + 1}/{NUM_FOLDS} ---")
    # Combined data splitting
    X_rest, X_new, y_rest, y_new = train_test_split(
        X, y, test_size=0.2, stratify=y, shuffle=True, random_state=repeat)

    X_train_personal_raw, X_test_personal_raw, y_train_personal, y_test_personal = train_test_split(
        X_new, y_new, test_size=0.2, stratify=y_new, random_state=42)

    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_rest, y_rest, test_size=0.2, stratify=y_rest, shuffle=True, random_state=repeat)

    # Preprocessing for central and new data
    X_train_scaled_central, scaler_central = preprocess_data(X_train_raw)
    X_test_scaled_central, _ = preprocess_data(X_test_raw, scaler=scaler_central)
    X_new_scaled, _ = preprocess_data(X_new)

    #Apply same preprocessing to personalized data
    X_train_personal_scaled, personal_scaler = preprocess_data(X_train_personal_raw)
    X_test_personal_scaled, _ = preprocess_data(X_test_personal_raw, scaler=personal_scaler)

    X_train_partitions_raw = np.array_split(X_train_raw, NUM_PARTITIONS)
    y_train_partitions = np.array_split(y_train, NUM_PARTITIONS)
    X_test_partitions_raw = np.array_split(X_test_raw, NUM_PARTITIONS)
    y_test_partitions = np.array_split(y_test, NUM_PARTITIONS)
    client_train_data = {}
    client_val_data = {}

    for cid in range(NUM_PARTITIONS):
        X_train_reduced = fit_bspline_basis(X_train_partitions_raw[cid], num_basis=basisnum, spline_order=3)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_reduced)

        X_test_reduced = fit_bspline_basis(X_test_partitions_raw[cid], num_basis=basisnum, spline_order=3)
        X_test_scaled = scaler.transform(X_test_reduced)

        client_train_data[cid] = (X_train_scaled, y_train_partitions[cid])
        client_val_data[cid] = (X_test_scaled, y_test_partitions[cid])

    run_simulation(
        server_app=server,
        client_app=client,
        num_supernodes=NUM_PARTITIONS,
        backend_config=backend_config
    )

    model = initialize_model(X_new_scaled.shape[1])
    set_parameters(model, final_parameters)
    model.classes_ = np.unique(y_new)
    final_fl_metrics = evaluate_model(model, X_new_scaled, y_new)

    def run_personalized_evaluation(final_parameters):
        model = initialize_model(X_train_personal_scaled.shape[1])
        set_parameters(model, final_parameters)
        # Train on train split
        model.partial_fit(X_train_personal_scaled, y_train_personal, classes=np.unique(y))
        # Evaluate on test split
        metrics = evaluate_model(model, X_test_personal_scaled, y_test_personal)
        return metrics

    personalized_metrics = run_personalized_evaluation(final_parameters)

    local_metrics_list = []
    local_models = []

    for cid in range(NUM_PARTITIONS):
        model = SGDClassifier(
            loss="log_loss", max_iter=200, tol=None,
            warm_start=True, learning_rate="optimal", eta0=0.01
        )
        X_train_client, y_train_client = client_train_data[cid]
        # Train local model on client data
        train_model(model, X_train_client, y_train_client)
        local_models.append(model)
        # Evaluate local model on the common new data
        metrics = evaluate_model(model, X_new_scaled, y_new)
        local_metrics_list.append(metrics)

    # Average metrics from local models evaluated on X_new_scaled
    local_metrics = pd.DataFrame(local_metrics_list).mean().to_dict()

    # Now average the local models' weights
    def average_sgd_models(models):
        # This assumes sklearn SGDClassifier, average coef_ and intercept_
        avg_model = initialize_model(X_new_scaled.shape[1])

        coefs = np.array([m.coef_ for m in models])
        intercepts = np.array([m.intercept_ for m in models])

        avg_coef = np.mean(coefs, axis=0)
        avg_intercept = np.mean(intercepts, axis=0)

        avg_model.coef_ = avg_coef
        avg_model.intercept_ = avg_intercept
        avg_model.classes_ = models[0].classes_

        return avg_model

    local_avg_model = average_sgd_models(local_models)
    local_avg_metrics = evaluate_model(local_avg_model, X_new_scaled, y_new)


    central_model = SGDClassifier(
        loss="log_loss", max_iter=200, tol=None,
        warm_start=True, learning_rate="optimal", eta0=0.01
    )
    train_model(central_model, X_train_scaled_central, y_train)
    central_metrics = evaluate_model(central_model, X_new_scaled, y_new)

    all_results = [
        {"mode": "federated", **final_fl_metrics},
        {"mode": "personalized_federated", **personalized_metrics},
        {"mode": "local", **local_metrics},
        {"mode": "local_avg", **local_avg_metrics},
        {"mode": "centralized", **central_metrics},
    ]

    for result in all_results:
        mode = result["mode"]
        results[mode].append(result)

    fold += 1

summary_rows = []
for mode, metrics_list in results.items():
    df_mode = pd.DataFrame(metrics_list)
    row = {"mode": mode}
    for metric in df_mode.columns:
        if metric == "mode":
            continue
        row[f"{metric}_mean"] = df_mode[metric].mean()
        row[f"{metric}_std"] = df_mode[metric].std()
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(f"results_20cv_basis{basisnum}_LR.csv", index=False)
print(summary_df)

# Save all results
all_results_by_basis[basisnum] = results.copy()

results = {
    "federated": [],
    "personalized_federated": [],
    "local": [],
    "local_avg": [],
    "centralized": []
}
fold = 0

##Visualizing Model Accuracy Across Modes for LR
This code aggregates accuracy across all cross-validation folds and model types, then visualizes the results.

- Collects accuracy scores from all modes and basis numbers into a DataFrame.
- Saves the results as LR_accuracies.csv.
- Plots a boxplot of accuracy by mode (e.g., federated, local, centralized) using Seaborn to compare performance distributions.  

This allows easy comparison of how each model type performs across different data splits and experiments.

In [ ]:
df_all = []
for basisnum, mode_results in all_results_by_basis.items():
    for mode, metrics_list in mode_results.items():
        for fold_metrics in metrics_list:
            df_all.append({
                "basis": basisnum,
                "mode": mode,
                "accuracy": fold_metrics["accuracy"]
            })

df_plot = pd.DataFrame(df_all)
df_plot.to_csv("LR_accuracies.csv", index=False)
print(df_plot)

# Plot accuracy distribution by mode
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_plot, x="mode", y="accuracy", hue="mode", palette="Set3")
plt.title("Accuracy Distribution Across Folds by Mode (LR)")
plt.xlabel("Mode")
plt.ylabel("Accuracy")
plt.ylim(0.5, 1.0)
plt.grid(axis="y")
plt.tight_layout()
plt.savefig("accuracy_boxplot_by_mode_LR.png", dpi=600)
plt.show()

##Support Vector Machine Setup

This code sets up a basic federated learning workflow using the Flower library and scikit-learn's SGDClassifier (hinge loss).

Parameter management: Functions get, set, and average the model's weights across different clients.  
Model utilities: Functions for incremental training (partial_fit) and evaluating with standard metrics (loss, accuracy, precision, recall, F1).  
FlowerClient: Each client handles its own data partition, receives and updates model parameters, trains locally, and reports metrics.  
Server functions: Aggregate client metrics, perform federated averaging, and orchestrate rounds.  
Configuration: Specifies number of rounds and system resources for clients.

In [ ]:
def get_parameters(model):
    return [model.coef_.copy(), model.intercept_.copy()]

def set_parameters(model, parameters):
    model.coef_ = parameters[0].copy()
    model.intercept_ = parameters[1].copy()

def average_parameters(parameter_list):
    avg_coef = np.mean([params[0] for params in parameter_list], axis=0)
    avg_intercept = np.mean([params[1] for params in parameter_list], axis=0)
    return [avg_coef, avg_intercept]

def train_model(model, X, y):
    if not hasattr(model, "coef_"):
        model.partial_fit(X, y, classes=np.array([0,1]))
    else:
        model.partial_fit(X, y)

def evaluate_model(model, X, y):
    preds = model.predict(X)
    decision_fn = model.decision_function(X)
    return {
        "loss": hinge_loss(y, decision_fn, labels=model.classes_),
        "accuracy": accuracy_score(y, preds),
        "precision": precision_score(y, preds, average="binary", zero_division=0),
        "recall": recall_score(y, preds, average="binary", zero_division=0),
        "f1": f1_score(y, preds, average="binary", zero_division=0),
    }
def initialize_model(n_features):
    model = SGDClassifier(
        loss="hinge",
        max_iter=20,
        tol=None,
        warm_start=True,
        learning_rate="optimal",
        eta0=0.01,
        random_state=42,
    )
    return model

class FlowerClient(NumPyClient):
    def __init__(self, partition_id: int, personalized=False):
        self.partition_id = partition_id
        self.model = initialize_model(X_train_full.shape[1])
        self.X_train, self.y_train = client_train_data[partition_id]
        self.X_val, self.y_val = client_val_data[partition_id]
        self.personalized = personalized
        self.model.partial_fit(self.X_train, self.y_train, classes=np.array([0, 1]))


    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        train_model(self.model, self.X_train, self.y_train)
        return get_parameters(self.model), len(self.X_train), {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        metrics = evaluate_model(self.model, self.X_val, self.y_val)
        loss = metrics.pop("loss", 0.0)
        return loss, len(self.y_val), metrics

final_fl_metrics = None
final_parameters = None

def get_evaluate_fn():
    def evaluate(server_round, parameters, config):
        all_metrics = []
        for cid in range(NUM_PARTITIONS):
            model = initialize_model(X_train_full.shape[1])
            X_val, y_val = client_val_data[cid]
            set_parameters(model, parameters)
            model.classes_ = np.unique(y_train_full)
            metrics = evaluate_model(model, X_val, y_val)
            all_metrics.append(metrics)

        df_metrics = pd.DataFrame(all_metrics)
        avg_metrics = df_metrics.mean().to_dict()

        if server_round == 10:
            global final_fl_metrics, final_parameters
            final_fl_metrics = avg_metrics.copy()
            final_parameters = parameters

        return avg_metrics["loss"], {"accuracy": avg_metrics["accuracy"]}
    return evaluate

def server_fn(context):
    strategy = FedAvg(
        min_available_clients=NUM_PARTITIONS,
        evaluate_fn=get_evaluate_fn(),
        fit_metrics_aggregation_fn=None,
        evaluate_metrics_aggregation_fn=None
    )
    return ServerAppComponents(config=ServerConfig(num_rounds=10), strategy=strategy)

def client_fn(context):
    cid = context.node_config.get("partition-id", 0)
    return FlowerClient(cid).to_client()

client = ClientApp(client_fn=client_fn)
server = ServerApp(server_fn=server_fn)

backend_config = {"client_resources": {"num_cpus": 2, "num_gpus": 0}}

##Cross-Validation Evaluation of Federated and Baseline Models for SVM
This script runs repeated train/test splits to compare federated, personalized, local, and centralized models.

- For each fold, the data is split into training, testing, and personalized subsets with stratification.
- Each partition is preprocessed and used to prepare client data.
- A federated simulation runs, aggregating client models.
- Evaluations are performed for:
federated learning (global model),
personalized federated (fine-tuned global model),
local-only models (trained on each client),
averaged local models (ensemble of locals),
and a centralized model (trained on pooled data).
- All results and summary statistics are saved for further analysis.

In [ ]:
results = {
    "federated": [],
    "personalized_federated": [],
    "local": [],
    "local_avg": [],
    "centralized": []
}

all_results_by_basis = {}
fold = 0

print(f"--- Basis {basisnum} ---")
for repeat in range(NUM_FOLDS):
    print(f"--- Fold {repeat + 1}/{NUM_FOLDS} ---")
    # Combined data splitting
    X_rest, X_new, y_rest, y_new = train_test_split(
        X, y, test_size=0.2, stratify=y, shuffle=True, random_state=repeat)

    X_train_personal_raw, X_test_personal_raw, y_train_personal, y_test_personal = train_test_split(
        X_new, y_new, test_size=0.2, stratify=y_new, random_state=42)

    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_rest, y_rest, test_size=0.2, stratify=y_rest, shuffle=True, random_state=repeat)

    # Preprocessing for central and new data
    X_train_scaled_central, scaler_central = preprocess_data(X_train_raw)
    X_test_scaled_central, _ = preprocess_data(X_test_raw, scaler=scaler_central)
    X_new_scaled, _ = preprocess_data(X_new)

    # Apply same preprocessing to personalized data
    X_train_personal_scaled, personal_scaler = preprocess_data(X_train_personal_raw)
    X_test_personal_scaled, _ = preprocess_data(X_test_personal_raw, scaler=personal_scaler)

    X_train_partitions_raw = np.array_split(X_train_raw, NUM_PARTITIONS)
    y_train_partitions = np.array_split(y_train, NUM_PARTITIONS)
    X_test_partitions_raw = np.array_split(X_test_raw, NUM_PARTITIONS)
    y_test_partitions = np.array_split(y_test, NUM_PARTITIONS)
    client_train_data = {}
    client_val_data = {}

    for cid in range(NUM_PARTITIONS):
        X_train_reduced = fit_bspline_basis(X_train_partitions_raw[cid], num_basis=basisnum, spline_order=3)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_reduced)

        X_test_reduced = fit_bspline_basis(X_test_partitions_raw[cid], num_basis=basisnum, spline_order=3)
        X_test_scaled = scaler.transform(X_test_reduced)

        client_train_data[cid] = (X_train_scaled, y_train_partitions[cid])
        client_val_data[cid] = (X_test_scaled, y_test_partitions[cid])

    run_simulation(
        server_app=server,
        client_app=client,
        num_supernodes=NUM_PARTITIONS,
        backend_config=backend_config
    )

    model = initialize_model(X_new_scaled.shape[1])
    set_parameters(model, final_parameters)
    model.classes_ = np.unique(y_new)
    final_fl_metrics = evaluate_model(model, X_new_scaled, y_new)

    def run_personalized_evaluation(final_parameters):
        model = initialize_model(X_train_personal_scaled.shape[1])
        set_parameters(model, final_parameters)
        # Train on train split
        model.partial_fit(X_train_personal_scaled, y_train_personal, classes=np.unique(y))
        # Evaluate on test split
        metrics = evaluate_model(model, X_test_personal_scaled, y_test_personal)
        return metrics

    personalized_metrics = run_personalized_evaluation(final_parameters)

    local_metrics_list = []
    local_models = []

    for cid in range(NUM_PARTITIONS):
        model = SGDClassifier(
            loss="hinge", max_iter=200, tol=None,
            warm_start=True, learning_rate="optimal", eta0=0.01
        )
        X_train_client, y_train_client = client_train_data[cid]
        # Train local model on client data
        train_model(model, X_train_client, y_train_client)
        local_models.append(model)
        # Evaluate local model on the common new data
        metrics = evaluate_model(model, X_new_scaled, y_new)
        local_metrics_list.append(metrics)

    # Average metrics from local models evaluated on X_new_scaled
    local_metrics = pd.DataFrame(local_metrics_list).mean().to_dict()

    # Now average the local models' weights
    def average_sgd_models(models):
        # This assumes sklearn SGDClassifier, average coef_ and intercept_
        avg_model = initialize_model(X_new_scaled.shape[1])

        coefs = np.array([m.coef_ for m in models])
        intercepts = np.array([m.intercept_ for m in models])

        avg_coef = np.mean(coefs, axis=0)
        avg_intercept = np.mean(intercepts, axis=0)

        avg_model.coef_ = avg_coef
        avg_model.intercept_ = avg_intercept
        avg_model.classes_ = models[0].classes_

        return avg_model

    local_avg_model = average_sgd_models(local_models)
    local_avg_metrics = evaluate_model(local_avg_model, X_new_scaled, y_new)


    central_model = SGDClassifier(
        loss="hinge", max_iter=200, tol=None,
        warm_start=True, learning_rate="optimal", eta0=0.01
    )
    train_model(central_model, X_train_scaled_central, y_train)
    central_metrics = evaluate_model(central_model, X_new_scaled, y_new)

    all_results = [
        {"mode": "federated", **final_fl_metrics},
        {"mode": "personalized_federated", **personalized_metrics},
        {"mode": "local", **local_metrics},
        {"mode": "local_avg", **local_avg_metrics},
        {"mode": "centralized", **central_metrics},
    ]

    for result in all_results:
        mode = result["mode"]
        results[mode].append(result)

    fold += 1

summary_rows = []
for mode, metrics_list in results.items():
    df_mode = pd.DataFrame(metrics_list)
    row = {"mode": mode}
    for metric in df_mode.columns:
        if metric == "mode":
            continue
        row[f"{metric}_mean"] = df_mode[metric].mean()
        row[f"{metric}_std"] = df_mode[metric].std()
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(f"results_20cv_basis{basisnum}_SVM.csv", index=False)
print(summary_df)

# Save all results
all_results_by_basis[basisnum] = results.copy()

results = {
    "federated": [],
    "personalized_federated": [],
    "local": [],
    "local_avg": [],
    "centralized": []
}
fold = 0

##Visualizing Model Accuracy Across Modes for SVM
This code aggregates accuracy across all cross-validation folds and model types, then visualizes the results.

- Collects accuracy scores from all modes and basis numbers into a DataFrame.
- Saves the results as SVM_accuracies.csv.
- Plots a boxplot of accuracy by mode (e.g., federated, local, centralized) using Seaborn to compare performance distributions.  

This allows easy comparison of how each model type performs across different data splits and experiments.

In [ ]:
df_all = []
for basisnum, mode_results in all_results_by_basis.items():
    for mode, metrics_list in mode_results.items():
        for fold_metrics in metrics_list:
            df_all.append({
                "basis": basisnum,
                "mode": mode,
                "accuracy": fold_metrics["accuracy"]
            })

df_plot = pd.DataFrame(df_all)
df_plot.to_csv("SVM_accuracies.csv", index=False)
print(df_plot)

# Plot accuracy distribution by mode
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_plot, x="mode", y="accuracy", hue="mode", palette="Set3")
plt.title("Accuracy Distribution Across Folds by Mode (SVM)")
plt.xlabel("Mode")
plt.ylabel("Accuracy")
plt.ylim(0.5, 1.0)
plt.grid(axis="y")
plt.tight_layout()
plt.savefig("accuracy_boxplot_by_mode_SVM.png", dpi=600)
plt.show()

## Neural Network Federated Learning Setup

This code sets up a federated learning workflow using the Flower library and a simple PyTorch neural network.

Model architecture: Defines a feedforward neural network with two hidden layers, batch normalization, dropout, and ReLU activations.  
Parameter management: Functions to get and set model weights as numpy arrays for communication between clients and server.  
Model utilities: Training uses Adam optimizer and cross-entropy loss; evaluation computes accuracy, precision, recall, and F1.  
FlowerClient: Each client manages its own data, updates its neural network, and shares metrics after local training.  
Server functions: Aggregate client metrics, perform federated averaging, orchestrate training rounds, and save final parameters.

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.model(x)

def get_parameters(model):
    return [val.cpu().numpy() for val in model.state_dict().values()]

def set_parameters(model, parameters):
    state_dict = model.state_dict()
    for k, v in zip(state_dict.keys(), parameters):
        state_dict[k] = torch.tensor(v)
    model.load_state_dict(state_dict, strict=True)

def train_model(model, X, y, epochs=15):
    model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y, dtype=torch.long).to(device)
    for _ in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()

def evaluate_model(model, X, y):
    model.to(device)
    model.eval()
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y, dtype=torch.long).to(device)
    with torch.no_grad():
        logits = model(X_tensor)
        preds = torch.argmax(logits, dim=1)
        probs = torch.softmax(logits, dim=1)
        loss = nn.CrossEntropyLoss()(logits, y_tensor).item()
    return {
        "loss": loss,
        "accuracy": accuracy_score(y, preds.cpu().numpy()),
        "precision": precision_score(y, preds.cpu().numpy(), average="binary", zero_division=0),
        "recall": recall_score(y, preds.cpu().numpy(), average="binary", zero_division=0),
        "f1": f1_score(y, preds.cpu().numpy(), average="binary", zero_division=0)
    }

class FlowerClient(NumPyClient):
    def __init__(self, partition_id: int, personalized=False):
        self.partition_id = partition_id
        self.model = SimpleNN(input_size=client_train_data[partition_id][0].shape[1])
        self.X_train, self.y_train = client_train_data[partition_id]
        self.X_val, self.y_val = client_val_data[partition_id]
        self.personalized = personalized

    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        train_model(self.model, self.X_train, self.y_train, epochs=15)
        return get_parameters(self.model), len(self.X_train), {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        X_val, y_val = client_val_data[self.partition_id]
        metrics = evaluate_model(self.model, X_val, y_val)
        loss = metrics.pop("loss")
        return loss, len(y_val), metrics

final_metrics = {}
final_fl_metrics = None
final_parameters = None

def get_evaluate_fn():
    def evaluate(server_round, parameters, config):
        all_metrics = []
        for cid in range(NUM_PARTITIONS):
            model = SimpleNN(input_size=client_val_data[cid][0].shape[1])
            X_val, y_val = client_val_data[cid]
            set_parameters(model, parameters)
            metrics = evaluate_model(model, X_val, y_val)
            all_metrics.append(metrics)

        avg_metrics = pd.DataFrame(all_metrics).mean().to_dict()

        if server_round == 10:  # final round
            global final_fl_metrics, final_parameters
            final_fl_metrics = avg_metrics.copy()
            final_parameters = parameters

        return avg_metrics["loss"], {"accuracy": avg_metrics["accuracy"]}
    return evaluate

def server_fn(context):
    strategy = FedAvg(
        min_available_clients=NUM_PARTITIONS,
        evaluate_fn=get_evaluate_fn(),
    )
    config = ServerConfig(num_rounds=10)
    return ServerAppComponents(config=config, strategy=strategy)

# === Client Factory ===
def client_fn(context):
    partition_id = context.node_config.get("partition-id", 0)
    return FlowerClient(partition_id).to_client()

client = ClientApp(client_fn=client_fn)
server = ServerApp(server_fn=server_fn)
backend_config = {"client_resources": {"num_cpus": 2, "num_gpus": 1}}

##Cross-Validation Evaluation of Federated and Baseline Models for NN
This script runs repeated train/test splits to compare federated, personalized, local, and centralized models.

- For each fold, the data is split into training, testing, and personalized subsets with stratification.
- Each partition is preprocessed and used to prepare client data.
- A federated simulation runs, aggregating client models.
- Evaluations are performed for:
federated learning (global model),
personalized federated (fine-tuned global model),
local-only models (trained on each client),
averaged local models (ensemble of locals),
and a centralized model (trained on pooled data).
- All results and summary statistics are saved for further analysis.

In [ ]:
results = {
    "federated": [],
    "personalized_federated": [],
    "local": [],
    "local_avg": [],
    "centralized": []
}

basisnum = 10

all_results_by_basis = {}
fold = 0

print(f"--- Basis {basisnum} ---")
for repeat in range(NUM_FOLDS):
    print(f"--- Fold {repeat + 1}/{NUM_FOLDS} ---")
    # Combined data splitting
    X_rest, X_new, y_rest, y_new = train_test_split(
        X, y, test_size=0.2, stratify=y, shuffle=True, random_state=repeat)

    X_train_personal_raw, X_test_personal_raw, y_train_personal, y_test_personal = train_test_split(
        X_new, y_new, test_size=0.2, stratify=y_new, random_state=42)

    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_rest, y_rest, test_size=0.2, stratify=y_rest, shuffle=True, random_state=repeat)

    # Preprocessing for central and new data
    X_train_scaled_central, scaler_central = preprocess_data(X_train_raw)
    X_test_scaled_central, _ = preprocess_data(X_test_raw, scaler=scaler_central)
    X_new_scaled, _ = preprocess_data(X_new)

    # Apply same preprocessing to personalized data
    X_train_personal_scaled, personal_scaler = preprocess_data(X_train_personal_raw)
    X_test_personal_scaled, _ = preprocess_data(X_test_personal_raw, scaler=personal_scaler)

    X_train_partitions_raw = np.array_split(X_train_raw, NUM_PARTITIONS)
    y_train_partitions = np.array_split(y_train, NUM_PARTITIONS)
    X_test_partitions_raw = np.array_split(X_test_raw, NUM_PARTITIONS)
    y_test_partitions = np.array_split(y_test, NUM_PARTITIONS)
    client_train_data = {}
    client_val_data = {}

    for cid in range(NUM_PARTITIONS):
        X_train_reduced = fit_bspline_basis(X_train_partitions_raw[cid], num_basis=basisnum, spline_order=3)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_reduced)

        X_test_reduced = fit_bspline_basis(X_test_partitions_raw[cid], num_basis=basisnum, spline_order=3)
        X_test_scaled = scaler.transform(X_test_reduced)

        client_train_data[cid] = (X_train_scaled, y_train_partitions[cid])
        client_val_data[cid] = (X_test_scaled, y_test_partitions[cid])

    run_simulation(
        server_app=server,
        client_app=client,
        num_supernodes=NUM_PARTITIONS,
        backend_config=backend_config
    )

    model = SimpleNN(input_size=X_new_scaled.shape[1])
    set_parameters(model, final_parameters)
    final_fl_metrics = evaluate_model(model, X_new_scaled, y_new)

    def run_personalized_evaluation(final_parameters):
        model = SimpleNN(input_size=X_new_scaled.shape[1])
        set_parameters(model, final_parameters)

        train_model(model, X_train_personal_scaled, y_train_personal, epochs=15)
        metrics = evaluate_model(model, X_test_personal_scaled, y_test_personal)
        return metrics

    personalized_metrics = run_personalized_evaluation(final_parameters)

    local_models = []
    local_metrics_list = []

    for cid in range(NUM_PARTITIONS):
        X_train_client, y_train_client = client_train_data[cid]
        model = SimpleNN(input_size=X_train_client.shape[1])
        train_model(model, X_train_client, y_train_client, epochs=150)

        local_models.append(model)

        metrics = evaluate_model(model, X_new_scaled, y_new)
        local_metrics_list.append(metrics)

    local_metrics = pd.DataFrame(local_metrics_list).mean().to_dict()

    def average_nn_models(models):
        avg_model = SimpleNN(input_size=X_new_scaled.shape[1])

        state_dicts = [m.state_dict() for m in models]
        avg_state_dict = {}

        for key in state_dicts[0]:
            avg_state_dict[key] = sum(d[key] for d in state_dicts) / len(state_dicts)

        avg_model.load_state_dict(avg_state_dict)
        return avg_model

    local_avg_model = average_nn_models(local_models)
    local_avg_metrics = evaluate_model(local_avg_model, X_new_scaled, y_new)

    central_model = SimpleNN(input_size=X_train_scaled_central.shape[1])
    train_model(central_model, X_train_scaled_central, y_train, epochs=150)
    central_metrics = evaluate_model(central_model, X_new_scaled, y_new)

    all_results = [
        {"mode": "federated", **final_fl_metrics},
        {"mode": "personalized_federated", **personalized_metrics},
        {"mode": "local", **local_metrics},
        {"mode": "local_avg", **local_avg_metrics},
        {"mode": "centralized", **central_metrics},
    ]

    for result in all_results:
        mode = result["mode"]
        results[mode].append(result)

    fold += 1

summary_rows = []
for mode, metrics_list in results.items():
    df_mode = pd.DataFrame(metrics_list)
    row = {"mode": mode}
    for metric in df_mode.columns:
        if metric == "mode":
            continue
        row[f"{metric}_mean"] = df_mode[metric].mean()
        row[f"{metric}_std"] = df_mode[metric].std()
    summary_rows.append(row)

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(f"results_20cv_basis{basisnum}_NN.csv", index=False)
print(summary_df)

# Save all results
all_results_by_basis[basisnum] = results.copy()

results = {
    "federated": [],
    "personalized_federated": [],
    "local": [],
    "local_avg": [],
    "centralized": []
}
fold = 0

##Visualizing Model Accuracy Across Modes for NN
This code aggregates accuracy across all cross-validation folds and model types, then visualizes the results.

- Collects accuracy scores from all modes and basis numbers into a DataFrame.
- Saves the results as NN_accuracies.csv.
- Plots a boxplot of accuracy by mode (e.g., federated, local, centralized) using Seaborn to compare performance distributions.  

This allows easy comparison of how each model type performs across different data splits and experiments.

In [ ]:
df_all = []
for basisnum, mode_results in all_results_by_basis.items():
    for mode, metrics_list in mode_results.items():
        for fold_metrics in metrics_list:
            df_all.append({
                "basis": basisnum,
                "mode": mode,
                "accuracy": fold_metrics["accuracy"]
            })

df_plot = pd.DataFrame(df_all)
df_plot.to_csv("NN_accuracies.csv", index=False)
print(df_plot)

# Plot accuracy distribution by mode
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_plot, x="mode", y="accuracy", hue="mode", palette="Set3")
plt.title("Accuracy Distribution Across Folds by Mode (NN)")
plt.xlabel("Mode")
plt.ylabel("Accuracy")
plt.ylim(0.5, 1.0)
plt.grid(axis="y")
plt.tight_layout()
plt.savefig("accuracy_boxplot_by_mode_NN.png", dpi=600)
plt.show()